- Implement an LCEL chain with a memory that uses one of the memory classes we learned about.  
  앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.

- The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").  
  이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").

- Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.  
  항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
- To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.  
  메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [2]:
examples = [
    {"movie": "Top Gun", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "The Godfather", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "Titanic", "answer": "🚢💔🌊"},
    {"movie": "Jurassic Park", "answer": "🦖🦕🏞️"},
    {"movie": "Frozen", "answer": "❄️👭⛄"},
    {"movie": "The Avengers", "answer": "🛡️💥🦸‍♂️"},
]

In [3]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True,
)

In [4]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)

fewshot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
               You are a smart assistant. 
               - If the input is a question, provide a detailed and informative answer.
               - If the input is a movie title, respond with exactly three emojis that best represent the movie, based on the provided examples.
               Do not include any additional text or explanation when responding to movie titles.
            """,
        ),
        fewshot_prompt,
        MessagesPlaceholder(variable_name="chat_history"),
        # ("human", "{movie}"),
        ("human", "{movie_or_question}"),  # LCEL
    ]
)

In [25]:
print(fewshot_prompt.format())

Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️


In [24]:
print(final_prompt.format(movie="Finding Nemo", chat_history=[]))

System: You are a movie emoji generator. When given a movie title as input, respond with exactly three emojis that best represent the movie, based on the provided examples. Do not include any additional text or explanation in your response.
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Finding Nemo


In [16]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=final_prompt,
    verbose=True,
)

In [26]:
chain.predict(movie="Inception")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a movie emoji generator. When given a movie title as input, respond with exactly three emojis that best represent the movie, based on the provided examples. Do not include any additional text or explanation in your response.
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Inception
💤🌀🕰️
> Finished chain.


'💤🌀🕰️'

In [27]:
chain.predict(movie="Star Wars")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a movie emoji generator. When given a movie title as input, respond with exactly three emojis that best represent the movie, based on the provided examples. Do not include any additional text or explanation in your response.
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Inception
AI: 💤🌀🕰️
Human: Star Wars
🌌⚔️🚀
> Finished chain.


'🌌⚔️🚀'

In [28]:
memory.chat_memory

ChatMessageHistory(messages=[HumanMessage(content='Inception'), AIMessage(content='💤🌀🕰️'), HumanMessage(content='Star Wars'), AIMessage(content='🌌⚔️🚀')])

In [29]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='Inception'),
 AIMessage(content='💤🌀🕰️'),
 HumanMessage(content='Star Wars'),
 AIMessage(content='🌌⚔️🚀')]

In [ ]:
chain.run(
    movie="Can you show me the emoji for the movie I asked about earlier?"
)  # examples에 있는 거 대답함



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a movie emoji generator. When given a movie title as input, respond with exactly three emojis that best represent the movie, based on the provided examples. Do not include any additional text or explanation in your response.
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Inception
AI: 💤🌀🕰️
Human: Star Wars
AI: 🌌⚔️🚀
Human: Can you show me the emoji for the movie I asked about earlier?
🛩️👨‍✈️🔥
> Finished chain.


'🛩️👨\u200d✈️🔥'

In [31]:
chain.predict(movie="What was the last movie I asked about?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a movie emoji generator. When given a movie title as input, respond with exactly three emojis that best represent the movie, based on the provided examples. Do not include any additional text or explanation in your response.
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Inception
AI: 💤🌀🕰️
Human: Star Wars
AI: 🌌⚔️🚀
Human: Can you show me the emoji for the movie I asked about earlier?
AI: 🛩️👨‍✈️🔥
Human: What was the last movie I asked about?
Star Wars
> Finished chain.
The human mentions "Inception," and the AI responds with emojis representing sleep, a spiral, and a clock.

'Star Wars'

In [32]:
memory.load_memory_variables({})["chat_history"]

[SystemMessage(content='The human mentions "Inception," and the AI responds with emojis representing sleep, a spiral, and a clock.'),
 HumanMessage(content='Star Wars'),
 AIMessage(content='🌌⚔️🚀'),
 HumanMessage(content='Can you show me the emoji for the movie I asked about earlier?'),
 AIMessage(content='🛩️👨\u200d✈️🔥'),
 HumanMessage(content='What was the last movie I asked about?'),
 AIMessage(content='Star Wars')]

# LCEL


In [5]:
memory.load_memory_variables({})["chat_history"]

[]

In [27]:
print(final_prompt.format(movie_or_question="Zootopia", chat_history=[]))

System: 
               You are a smart assistant. 
               - If the input is a question, provide a detailed and informative answer.
               - If the input is a movie title, respond with exactly three emojis that best represent the movie, based on the provided examples.
               Do not include any additional text or explanation when responding to movie titles.
            
Human: Top Gun
AI: 🛩️👨‍✈️🔥
Human: The Godfather
AI: 👨‍👨‍👦🔫🍝
Human: Titanic
AI: 🚢💔🌊
Human: Jurassic Park
AI: 🦖🦕🏞️
Human: Frozen
AI: ❄️👭⛄
Human: The Avengers
AI: 🛡️💥🦸‍♂️
Human: Zootopia


In [6]:
from langchain.schema.runnable import RunnablePassthrough


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"movie_or_question": question})

    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

    # print(result)

In [7]:
invoke_chain("Herry Potter")

⚡🧙‍♂️📚

In [8]:
invoke_chain("Finding Nemo")

🐠🔍🌊

In [11]:
invoke_chain("What was the last movie I asked about?")

The last movie you asked about was "Finding Nemo."

In [13]:
invoke_chain("The Lion King")

🦁👑🌅The human mentions "Herry Potter."

In [15]:
invoke_chain("Spider-Man")

🕷️🕸️🕺The human mentions "Herry Potter," and the AI responds with emojis related to magic and books. The human then brings up "Finding Nemo."

In [16]:
load_memory(_)

[SystemMessage(content='The human mentions "Herry Potter," and the AI responds with emojis related to magic and books. The human then brings up "Finding Nemo."'),
 AIMessage(content='🐠🔍🌊'),
 HumanMessage(content='What was the last movie I asked about?'),
 AIMessage(content='The last movie you asked about was "Finding Nemo."'),
 HumanMessage(content='The Lion King'),
 AIMessage(content='🦁👑🌅'),
 HumanMessage(content='Spider-Man'),
 AIMessage(content='🕷️🕸️🕺')]

In [28]:
# SystemMessage 가 같이 출력되는 이유...?
# 이모지로 물어보는 중간에 movie가 아닌 질문을 해서 형식이 깨진거라 생각 -> prompt 수정. {movie_or_question}. 근데 아닌듯